# **Yolo ONNX Runtime**
***

In [1]:
from ultralytics import YOLO
import numpy as np
import onnx
import cv2
from onnx_inference import Yolov8Seg
import time

***
**Model Export als ONNX**

In [2]:
model = YOLO(r"C:\Users\morit\Desktop\YoloV8_InstanceSeg\models\20250110_2023_yolo8nano_pruned.pt")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\morit\\Desktop\\YoloV8_InstanceSeg\\models\\20250110_2023_yolo8nano_pruned.pt'

In [4]:
model.export(format='onnx')

Ultralytics 8.3.59  Python-3.10.11 torch-2.5.1+cu124 CPU (AMD Ryzen 7 5800H with Radeon Graphics)

PyTorch: starting from 'C:\Users\morit\Desktop\YoloV8_InstanceSeg\models\20250110_2023_yolo8nano_pruned.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 40, 8400), (1, 32, 160, 160)) (6.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.47...
ONNX: export success  2.7s, saved as 'C:\Users\morit\Desktop\YoloV8_InstanceSeg\models\20250110_2023_yolo8nano_pruned.onnx' (12.7 MB)

Export complete (3.1s)
Results saved to C:\Users\morit\Desktop\YoloV8_InstanceSeg\models
Predict:         yolo predict task=segment model=C:\Users\morit\Desktop\YoloV8_InstanceSeg\models\20250110_2023_yolo8nano_pruned.onnx imgsz=640  
Validate:        yolo val task=segment model=C:\Users\morit\Desktop\YoloV8_InstanceSeg\models\20250110_2023_yolo8nano_pruned.onnx imgsz=640 data=C:\Users\morit\Desktop\YoloV8_InstanceSeg\dataset\YOLOv8Dataset\dataset.yaml  
Visua

'C:\\Users\\morit\\Desktop\\YoloV8_InstanceSeg\\models\\20250110_2023_yolo8nano_pruned.onnx'

***
**Laden und Inferenz**

In [3]:
onnx_path = r"/home/mo/YoloV8_InstanceSeg/models/20250110_2023_yolo8nano_pruned.onnx"
yolo_yaml_filepath = r"/home/mo/YoloV8_InstanceSeg/dataset/YOLOv8Dataset/dataset.yaml"

In [4]:
model = Yolov8Seg(onnx_model=onnx_path, yolo_yaml_filepath=yolo_yaml_filepath)

In [5]:
img_path = r"dataset/image_1735916671.png"
img = cv2.imread(img_path)

In [6]:
num_inferences = 1

time_values = []
pre_times = []
inf_times = []
post_times = []

for inf in range(num_inferences):
    start_time = time.perf_counter()

    boxes, segments, _, fc_time_values = model(img, conf_threshold=0.4, iou_threshold=0.7)

    end_time = time.perf_counter()
    time_values.append(end_time - start_time)

    pre_times.append(fc_time_values[0])
    inf_times.append(fc_time_values[1])
    post_times.append(fc_time_values[2])    

print(f"Durchschnittliche Laufzeit Fc-Call: {(sum(time_values) / len(time_values)):.4f} Sekunden\n")
print(f"Durchschnittliche Preprocessing-Laugzeit: {(sum(pre_times) / len(pre_times)):.4f} Sekunden")
print(f"Durchschnittliche Inferenz-Laugzeit: {(sum(inf_times) / len(inf_times)):.4f} Sekunden")
print(f"Durchschnittliche Postprocessing-Laugzeit: {(sum(post_times) / len(post_times)):.4f} Sekunden")

Durchschnittliche Laufzeit Fc-Call: 0.2544 Sekunden

Durchschnittliche Preprocessing-Laugzeit: 0.0029 Sekunden
Durchschnittliche Inferenz-Laugzeit: 0.1238 Sekunden
Durchschnittliche Postprocessing-Laugzeit: 0.1276 Sekunden


In [7]:
if len(boxes) > 0:
        model.draw_and_visualize(img, boxes, segments, vis=False, save=True)